# FIT5196 Task 1 in Assessment 1
### Student Name: HsinHui Lin
### Student ID: 28464176
### Date: 02/Sep/2018
### Version: 1.0
### Environment: Python 3.6 and Jupyter notebook
### Libraries used: 
### re (for regular expression, included in Anaconda Python 3.6)
### json (for parsing json from strings or files, included in Anaconda Python 3.6)


### ______
#### Import libraries

In [ ]:
import re
import json

#### Load data
#### open and read the file

In [ ]:
with open("28464176.dat") as file:
    data=file.read()
file.close

#### examining data--get the data from 0 to 99

In [ ]:
data[0:100]

#### split the data and change to the next line.

In [ ]:
lines=data.split("\n")

#### get the lines from 0 to 99

In [ ]:
lines[0:100]

### divide jobs
#### as we see above, the jobs is divided by '------------------------------'
#### then we check how many jobs is here

In [ ]:
jobs=data.split("------------------------------")
len(jobs)

In [ ]:
#We check the last two lines, if there is a "------------------------------" or a "\n",
#at the end of last job
print(lines[-1])
print(lines[-2])

#### Get the job from the last one

In [ ]:
jobs[-1]

#### Get the jobs form the last two

In [ ]:
jobs[-2]

#### Get the elements of the list from the last one

In [ ]:
jobs=jobs[:-1]

thus, there should be 32004-1 jobs to wrangling.

### Compile a regular expression pattern into a regular expression
#### Findall( ) function is used in returning all matching keywords
#### using set to avoid the repeat values

In [ ]:
pattern = re.compile(r'[\w]*[ ]*[\w]+:', re.I)
tags=pattern.findall(data.lower())
tag_set=set(tags)

#### create a loop and find the common tag which with over 50 frequency ; then append to the new list.

In [ ]:
tag_frequence={}
for tag in tag_set:
    tag_frequence[tag]=tags.count(tag)
    
#find the common tag which appears more than 50 times in all data
tag_to_select=[]
for t in tag_frequence.keys():
    if tag_frequence[t]>=50:
        tag_to_select.append(t.lower())

#### print out the result

In [ ]:
print(tag_to_select)

### select and build the json key manually

In [ ]:

json_key={"_id:":"id",\
          "salary:":"salary","job_sal:":"salary","remuneration:":"salary",\
          "title:":"title","titles:":"title","_ttl:":"title","job title:":"title",\
          "location:":"location","locations:":"location","_loc:":"location","_locs:":"location","job_loc:":"location",\
          "description:":"job_descriptions","_description:":"job_descriptions","job_desc:":"job_descriptions","job description:":"job_descriptions",\
          "responsibility:":"job_responsibilities","responsibilities:":"job_responsibilities","job responsibilities:":"job_responsibilities",\
          "job_resps:":"job_responsibilities","resp:":"job_responsibilities",\
          "qualification:": "required_qualifications","qualifications:":"required_qualifications","required qualifications:":"required_qualifications",\
          "desired qualifications:":"required_qualifications","req_quals:":"required_qualifications",\
          "procedure:":"application_procedure","procedures:":"application_procedure","job_proc:":"application_procedure","job_procs:":"application_procedure",\
          "start_date:":"start_date","start date:":"start_date","date_start:":"start_date","dates:":"start_date","start_da:":"start_date",\
          "dead_line:":"application_deadline","deadlines:":"application_deadline","application_deadl:":"application_deadline",\
          "about_company:":"about_company","about company:":"about_company","companys_info:":"about_company","_info:":"about_company","about:":"about_company"
         }
         

#### Use if statement to check the line ; the strip( ) removes elements as mentioned on the statement. find the json_key and if not return "not a start"

In [ ]:
def New_line_checker(line):
    if line.lower().strip().startswith("-"):
        return True,"-"
    if line.lower().strip().startswith("id:"):
        return True,"_id"
    for key in json_key.keys():
        if line.lower().strip().startswith(key):
            return True,json_key[key]
    return False,"not a start"

#### Create a list named lines_structured and loop the list ; with the condition matched the true then return the structured line.

In [ ]:
def Structure_jobs(job):
    lines=job.split("\n")
    lines_structured=[]
    for line in lines:
        if (len(line.strip())==0 or line.lower().endswith("/") or line.lower().startswith("other responsibilities:"))==False:
            if line.strip()[-1] in [".",";"]:
                line=line[:-1]
            if New_line_checker(line)[0]==False:
                lines_structured[-1]+=" "
                lines_structured[-1]+=line.strip()
            else:
                lines_structured.append(line.strip())
    return lines_structured

### The in-built function-map is to project or map one list to another

In [ ]:
jobs=[job for job in map(Structure_jobs,jobs)]

In [ ]:
### have a quick look for the structured job
jobs[4]

#### create a dictionary with empty value ; for example: {'title': 'N/A', 'application_procedure': 'N/A', 'job_descriptions': 'N/A', 'application_deadline': 'N/A', 'about_company': 'N/A', 'job_responsibilities': 'N/A', 'salary': 'N/A', 'required_qualifications': 'N/A', 'location': 'N/A', 'start_date': 'N/A', 'id': 'N/A'}

In [ ]:
job_dict={}
for value in set(json_key.values()):
    job_dict[value]="N/A"
print(job_dict)

#### build the dictionary with if-else statement to return the job_dict

In [ ]:
def Build_job_dic(job):
    job_dict={}
    for value in set(json_key.values()):
        job_dict[value]="N/A"
    del job_dict["id"]
    dealing=""
    sub=""
    #json_key
    for line in job:
        boolean,start=New_line_checker(line)
        if start=="_id":
              job_dict["_id"]=line.split(":",1)[1]
        else:                    
            if boolean==True and start!="-":
                dealing=""
                sub=""
                if len(line.split(":",1))>1 and len(line.split(":",1)[1].strip())==0:
                    job_dict[json_key[line.split(":",1)[0].lower()+":"]]="N/A"
                else:
                    job_dict[json_key[line.split(":",1)[0].lower()+":"]]=line.split(":",1)[1].lower().strip()
                if line.split(":",1)[1].strip() !="":
                    job_dict[json_key[line.split(":",1)[0].lower()+":"]]=line.split(":",1)[1].strip()
                if(json_key[line.split(":",1)[0].lower()+":"]=="job_responsibilities"):
                    dealing=json_key[line.split(":",1)[0].lower()+":"]
                    sub="responsibility"
                    job_dict["job_responsibilities"]={"responsibility":[]}
                if(json_key[line.split(":",1)[0].lower()+":"]=="required_qualifications"):
                    dealing=json_key[line.split(":",1)[0].lower()+":"]
                    sub="qualification" 
                    job_dict["required_qualifications"]={"qualification":[]}                
                if(json_key[line.split(":",1)[0].lower()+":"]=="job_descriptions"):
                    dealing=json_key[line.split(":",1)[0].lower()+":"]
                    sub="description"
                    job_dict["job_descriptions"]={"description":[]}
        if dealing!="" and start.startswith("-")==False:
            if len(line.split(":",1))>1 and len(line.split(":",1)[1].strip())!=0: 
                job_dict[dealing][sub].append(line.split(":",1)[1].strip())
        if boolean==True and (start.startswith("-")==True) and dealing!="":
            job_dict[dealing][sub].append(line.split("-",1)[1].strip())
    return job_dict

#### create job_info_dict with map function and refer the Build_job_dict

In [ ]:
job_info_dicts=[job_info for job_info in map(Build_job_dic,jobs)]

#### have a quick look of the structured job

In [ ]:
jobs[6]

#### build job dictionary and have a quick look

In [ ]:
Build_job_dic(jobs[6])

#### build xml and using if-else statement 

In [ ]:
def Build_xml(job_dict):    
    xml="<listing id=\"{}\">\n".format(job_dict["_id"].strip())
    for key in job_dict.keys():
        if key!="_id":
            if key not in ["job_descriptions","job_responsibilities","required_qualifications"]:
                xml=xml+"<{}>\n{}\n""</{}>\n".format(key,job_dict[key].strip().replace("&","&amp;").replace("<","&lt;")\
                                                           .replace(">","&gt;").replace("\"","&quot;").replace("'","&apos;"),key)
            if key in ["job_descriptions","job_responsibilities","required_qualifications"]:
                xml+="<{}>\n".format(key)
                sub=""
                if key=="job_descriptions":
                    sub="description"
                if key=="job_responsibilities":
                    sub="responsibility"
                if key=="required_qualifications":
                    sub="qualification"
                if job_dict[key]=="N/A":
                    xml+="N/A"
                else:
                    for value in job_dict[key][sub]:
                        xml+="<{}>\n{}\n""</{}>\n".format(sub,value.strip().replace("&","&amp;").replace("<","&lt;")\
                                                           .replace(">","&gt;").replace("\"","&quot;").replace("'","&apos;"),sub)
                xml+="</{}>\n".format(key)
    xml+="</listing>"
    xml+="\n"
    return xml

#### to project or map one list to another that match the format.

In [ ]:
job_xmls=[job_xml for job_xml in map(Build_xml,job_info_dicts)]

#### print out the result

In [ ]:
print(job_xmls[0])

#### create the loop to form the job_xml_final

In [ ]:
job_xml_final="<?xml version=\"1.0\" encoding=\"UTF-8\" ?>\n<listings>\n"
for job_xml in job_xmls:
    job_xml_final+=job_xml
job_xml_final+="</listings>\n"

#### check the format of xml string

In [ ]:
print(job_xml_final[0:3000])

#### xml_file 

In [ ]:
xml_file= open('28464176.xml', 'w')
xml_file.write(job_xml_final)
xml_file.close()

#### format the json_string

In [ ]:
json_string=json.dumps({"listings":{"listing":job_info_dicts}}, indent=2)

#### check the format of json string

In [ ]:
print(json_string[0:2000])

#### json_file

In [ ]:
json_file = open('28464176.json', 'w')
json_file.write(json_string)
json_file.close()